### Outstanding tasks
- remove retweets RT @
- convert html entities to their own
- remove hyperlinks
- remove \n
- remove tweet ID mentions
- remove non-English tweets
- write code to allow easier future presentation
- study NLP lectures to cover gaps in knwoledge
- GCP Google Cloud - COLAB, should be able to use it under Ryerson
- keep notebook diary 

11/11
- add readme file explaining rehydration process
- also explain anything that was done outside of python

# Preload setup

In [1]:
%config IPCompleter.greedy=True
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 100

# Import Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:


#df = pd.read_csv('Book1-fastsave.csv')
df = pd.read_csv("/content/drive/My Drive/__CIND 820 - Data Analytics Project/3-data/Book1-fastsave.csv")


- Setting display options
- previewing the text attribute

# Cleaning Data

Converting retweets id null to 0, and entire attribute as int64

In [4]:
df.retweet_id = df[df['retweet_id'].notnull()].retweet_id.astype('int64') 

In [5]:
df.retweet_id = df.retweet_id.fillna(0).astype('int64')
df.in_reply_to_status_id = df.in_reply_to_status_id.fillna(0).astype('int64')
df.in_reply_to_user_id = df.in_reply_to_user_id.fillna(0).astype('int64')

Search the text attribute for any URLs.

from text attribute
- removing \n
- remove URL
- remove user referrals
- remove hashtags

In [6]:
# df[df.text.str.contains('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', regex=True)].text

In [7]:
df['modified_text'] = df.text.str.replace(r'\n', '')
df['modified_text'] = df.modified_text.str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '')
df['modified_text'] = df.modified_text.str.replace(r'\B@\w+', '')
df['modified_text'] = df.modified_text.str.replace(r'\B#\w+', '')

# Splitting Dataframes (originals and retweets)




Since the collection of tweets consist of original tweets and retweets, we can split them into two dataframes to avoid wasting time running sentimental analysis on retweets, since retweets are a copy of the original retweet by someone other than the original author.


`dfOriginals` will consist of tweets that are original--unique tweets, and `dfRetweets` consists all tweets that are not original.


In [8]:
dfOriginals = df[df.retweet_id == 0]
dfRetweets = df[df.retweet_id != 0]

## Original Tweets

In [9]:
# dfOriginals.head(5)

## Retweets


In [10]:
# dfRetweets.head(5)

## Guessing language

In [11]:
#https://pypi.org/project/langdetect/
!pip install langdetect
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

# import multiprocessing as mp
# p=mp.Pool(4)


In [12]:
# p.map(detect,dfOriginals.sample(500).text)

In [13]:
dfOriginals = dfOriginals[~(dfOriginals.modified_text == "")]

In [14]:
dfOriginals[(dfOriginals.modified_text == "")]

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,modified_text


add markdown giving explanation

In [15]:
def is_en(txt):
    try:
        return detect(txt)=='en'
    except:
        return False

In [16]:
dfOriginals.sample(5).text.apply(is_en)

492816    True
620196    True
738864    True
305403    True
764598    True
Name: text, dtype: bool

In [17]:
#https://stackoverflow.com/questions/60930935/exclude-non-english-rows-in-pandas

dfOriginals['guessed_language'] = dfOriginals.modified_text.apply(is_en)

# split to cores
# dfOriginals['guessed_language'] = p.map(is_en,dfOriginals.modified_text)
#dfOriginals.sample(1000).text.apply(detect)

- give extra information the analysis process
- what action did to the data

In [18]:
sum(dfOriginals.guessed_language)

232816

In [19]:
len(dfOriginals)

245388

# Being Sentiment Analysis

Import required libraries

In [20]:
import nltk
from nltk.sentiment.util import *
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [21]:
sid = SentimentIntensityAnalyzer()

In [22]:
# for sentence in sentences:
#      print(sentence)
#      ss = sid.polarity_scores(sentence)
#      for k in sorted(ss):
#          print('{0}: {1}, '.format(k, ss[k]), end='')
#      print()

## Calculating Setimental Score



In [23]:
dfOriginals['sentimentscore'] = dfOriginals.text.apply(sid.polarity_scores)

Quick preview of the sentiment score and its text.

In [24]:
dfOriginals.sample(30)[['modified_text','sentimentscore']]

,modified_text,sentimentscore
792852,"My kids mothers are planing moving out of US if vaccine for corona is mandatory..I said, fuck yeah..","{'neg': 0.171, 'neu': 0.829, 'pos': 0.0, 'compound': -0.5423}"
198058,"CN Corona (obviously), capitalism - excellent essay about how we're not living in a zombie apocalypse","{'neg': 0.0, 'neu': 0.791, 'pos': 0.209, 'compound': 0.5719}"
566313,Because what’s all this noise.Fuck this corona. 🙂🙂,"{'neg': 0.259, 'neu': 0.741, 'pos': 0.0, 'compound': -0.5423}"
375286,After corona😂❤️,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
528604,if I were able to be osmosis jones I’d kick the living shit out of corona,"{'neg': 0.205, 'neu': 0.795, 'pos': 0.0, 'compound': -0.5574}"
531664,"Our Play Team decided to dress up in our own versions of PPE, to take a group picture honoring . Who do you think ""wore it best""?","{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compound': 0.6908}"
59223,"Why? cz d dead don't want corona in heaven? We need to sanitize their heads first. Result of Electing rich empty heads to leadership positions then expecting their empties to conjure solutions when their monies can't buy us out. Anyway they have a point I guess,but not priorities","{'neg': 0.165, 'neu': 0.684, 'pos': 0.151, 'compound': 0.2354}"
189591,Nahh forreal!,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
346466,@Gurmeetramrahim Serve humanity work by volunteersat that face all world corona virus (COVID- 19) a dangerous desease. which no antidote till now and all to participate in this humanity work .,"{'neg': 0.155, 'neu': 0.845, 'pos': 0.0, 'compound': -0.6486}"
142,"There have been nine new cases of coronavirus confirmed in Western Australia overnight, taking the state’s total to 364.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"


In [60]:
# dfOriginals.sentimentscore.head(5)
dfOriginalSScore = pd.json_normalize(dfOriginals.sentimentscore)
dfOriginalSScore['original_index'] = dfOriginals.index
dfOriginalSScore = dfOriginalSScore.set_index('original_index')

In [61]:
# dfOriginals.join(dfOriginalSScore)
dfOriginalSScore.head(5)

,neg,neu,pos,compound
original_index,,,,
3,0.106,0.750,0.144,0.0129
4,0.000,0.882,0.118,0.3400
6,0.000,0.828,0.172,0.4404
7,0.000,0.802,0.198,0.6369
12,0.053,0.690,0.258,0.8885


- need some explanation on the "compound" attribute
- have an answer what the compound score stands for.

In [62]:
# pd.merge(dfOriginals, dfOriginalSScore, left_index=True, right_index='original_index')
dfOriginals.merge(dfOriginalSScore, left_index=True, right_index=True)

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,modified_text,guessed_language,sentimentscore,neg,neu,pos,compound
3,NaN,Tue Mar 31 04:35:43 +0000 2020,NaN,NaN,NaN,0,1244845807926272001,IMBHAVESHG,1244695118029807616,1163408885769617408,en,NaN,NaN,0,0,NaN,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>","CORONA PATIENT INCREASE AFTER LOCKDOWN BECOZ PM MODIJI TAKE LATE DECESSION ABOUT LOCKDOWN NOT MODIJI BUT WHOLE WORLD TAKE LATE DECESSION ABOUT LOCKDOWN AND WORLD ALSO LATE TO TAKE STEP AGAIN CORONA WHEN CORONA CRISIS INCREASE IN OUR COUNTRY GOVT ENGAGE IN "" NAMSTE TRUMP """,https://twitter.com/IMBHAVESHG/status/1244845807926272001,Mon Aug 19 11:14:47 +0000 2019,IMBHAVESHG,False,think difference other than,330,54,172,0,"Ahmadabad City, India",BHAVESH GOVINDBHAI PARMAR,IMBHAVESHG,1938,NaN,NaN,False,"CORONA PATIENT INCREASE AFTER LOCKDOWN BECOZ PM MODIJI TAKE LATE DECESSION ABOUT LOCKDOWN NOT MODIJI BUT WHOLE WORLD TAKE LATE DECESSION ABOUT LOCKDOWN AND WORLD ALSO LATE TO TAKE STEP AGAIN CORONA WHEN CORONA CRISIS INCREASE IN OUR COUNTRY GOVT ENGAGE IN "" NAMSTE TRUMP """,True,"{'neg': 0.106, 'neu': 0.75, 'pos': 0.144, 'compound': 0.0129}",0.106,0.750,0.144,0.0129
4,NaN,Tue Mar 31 04:35:42 +0000 2020,NaN,NaN,NaN,0,1244845802993979392,NaN,0,0,en,NaN,NaN,0,0,NaN,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This whole Corona , social distancing thing is giving me a lick of life after I graduate and mannn . Start the countdown",https://twitter.com/solarspaxew/status/1244845802993979392,Mon Mar 09 03:06:11 +0000 2015,solarspaxew,False,everyone has a chapter they don’t read out loud but Twitter is the exception,1022,73,146,0,"Birmingham, AL",Cass🧑🏽‍🌾,solarspaxew,307,NaN,NaN,False,"This whole Corona , social distancing thing is giving me a lick of life after I graduate and mannn . Start the countdown",True,"{'neg': 0.0, 'neu': 0.882, 'pos': 0.118, 'compound': 0.34}",0.000,0.882,0.118,0.3400
6,NaN,Tue Mar 31 04:35:47 +0000 2020,NaN,NaN,NaN,0,1244845825357905920,bandaydaputtar,1244841540050022400,815567265269747712,en,NaN,NaN,0,0,NaN,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>","@bandaydaputtar It seems a good decision. Corona tigers, But Just Need to be implemented with precaution.",https://twitter.com/Junniibaba/status/1244845825357905920,Sun Sep 22 04:21:12 +0000 2013,Junniibaba,False,"I am an IT Student, Web Designer,Web Developer and Programmer in C,C#,Python.\nYouTube Videos Creator at #Junni4You Channel. Subscribe it for Future Concern.",1664,55,179,0,"Sindh, Pakistan",Junnii Baba,Junniibaba,1213,NaN,https://junni4you.blogspot.com/,False,"It seems a good decision. Corona tigers, But Just Need to be implemented with precaution.",True,"{'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'compound': 0.4404}",0.000,0.828,0.172,0.4404
7,NaN,Tue Mar 31 04:35:48 +0000 2020,NaN,NaN,https://twitter.com/realTuckFrumper/status/1244819496386342913,0,1244845830168891393,NaN,0,0,en,NaN,False,0,0,NaN,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Please tell me when he gets it. Send a pic of him on a ventilator. Or better yet. A pic of him waiting for a ventilator https://t.co/yCmyeZAgcq,https://twitter.com/PaulieFilmpfc/status/1244845830168891393,Thu May 03 12:23:38 +0000 2012,PaulieFilmpfc,False,NaN,1446,79,457,1,NaN,Paulie C,PaulieFilmpfc,3422,NaN,NaN,False,Please tell me when he gets it. Send a pic of him on a ventilator. Or better yet. A pic of him wait

- impact factor of a tweet 